In [ ]:
# Install required packages
%pip install strands-agents>=0.1.2 strands-agents-tools>=0.1.1 python-dotenv


In [ ]:
# Import the libraries we'll use throughout this tutorial
import os
from dotenv import load_dotenv
from strands import Agent

# Load environment variables (for API keys)
load_dotenv()

print("✅ Basic setup complete! Let's start building our first agent.")


In [ ]:
# Create our first basic startup advisor
basic_advisor = Agent(
    system_prompt="""You are a startup advisor with experience in market analysis and business strategy. 
    Help entrepreneurs think through their business ideas by asking good questions and providing practical advice.
    Be encouraging but realistic."""
)

# Test it with a simple query
startup_idea = "I want to start a food delivery app for healthy meals"

print("🤖 Basic Advisor says:")
response = basic_advisor(f"Give me initial thoughts on this startup idea: {startup_idea}")
print(response)


In [ ]:
# Import additional libraries for MCP connection
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Check if we have the API key
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

if PERPLEXITY_API_KEY:
    print("✅ Perplexity API key found!")
else:
    print("⚠️ Please set your PERPLEXITY_API_KEY in a .env file")
    print("You can still continue with the tutorial, but search features won't work")


In [ ]:
def create_perplexity_connection():
    """
    Creates an agent with web search capabilities.
    This is our first 'powered-up' agent!
    """
    if not PERPLEXITY_API_KEY:
        print("❌ Cannot create search agent without API key")
        return None
    
    # Set up the connection to Perplexity search service
    perplexity_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="docker",
                args=[
                    "run", "-i", "--rm", 
                    "-e", "PERPLEXITY_API_KEY",
                    "mcp/perplexity-ask"
                ],
                env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY},
            )
        )
    )
    
    return perplexity_client

# Test our connection
search_client = create_perplexity_connection()
if search_client:
    print("✅ Search client created successfully!")
    print("🔍 Our agent can now search the web for real-time information!")


In [ ]:
# Now let's create a research agent that can search the web
def test_search_agent():

    with search_client:
        # Get the search tools available
        search_tools = search_client.list_tools_sync()
        print(f"🛠️ Available search tools: {[tool.name for tool in search_tools]}")
        
        # Create a research agent with search capabilities
        research_agent = Agent(
            system_prompt="""You are a market researcher. Use web search to find current information 
            about markets, competitors, and industry trends. Provide factual, up-to-date insights.""",
            tools=search_tools
        )
        
        # Test it!
        query = "Search for information about the food delivery market size and main competitors"
        print(f"\n🔍 Researching: {query}")
        
        result = research_agent(query)
        print("\n📊 Research Results:")
        print(result)

# Run the test
test_search_agent()


In [ ]:
# Import the swarm tool
from strands_tools import swarm

def test_swarm_research():
    """
    Test how multiple agents can work together on research
    """
    if not search_client:
        print("⚠️ Skipping swarm test - no search capabilities")
        return
        
    with search_client:
        search_tools = search_client.list_tools_sync()
        
        # Create a lead researcher who can coordinate a team
        lead_researcher = Agent(
            system_prompt="""You are a Lead Market Researcher who coordinates research teams.
            Use the swarm tool to delegate different aspects of market research to specialist agents.
            You can create multiple agents to research competitors, market size, trends, etc.""",
            tools=search_tools + [swarm]  # Note: we're adding the swarm tool!
        )
        
        # Give it a complex research task
        research_task = """
        Research the food delivery market. I need:
        1. Market size and growth trends
        2. Main competitors and their strategies  
        3. Customer behavior and preferences
        
        Use a swarm of 3 agents to divide this work efficiently.
        """
        
        print("🐝 Lead Researcher coordinating swarm...")
        print(f"📋 Task: {research_task}")
        
        result = lead_researcher(research_task)
        print("\n" + "="*60)
        print("🔍 SWARM RESEARCH RESULTS")
        print("="*60)
        print(result)

# Test the swarm
test_swarm_research()


## Step 5: Specialized Roles - The Strategy Team

Now we'll create specialized agents with different roles: a Market Analyst and a Strategic Advisor. Each will have a specific expertise and personality.

**Key Concept**: Different agents can have different "personalities" and expertise areas, just like hiring specialists for different roles in your company.


In [ ]:
def create_strategy_team():
    """
    Create our specialized strategy team with different roles
    """
    if not search_client:
        print("⚠️ Cannot create strategy team without search capabilities")
        return None, None
        
    with search_client:
        search_tools = search_client.list_tools_sync()
        
        # Market Analyst - focuses on data and research
        market_analyst = Agent(
            system_prompt="""You are a Market Analyst at a top consulting firm. 
            Your expertise is in gathering and analyzing market data, competitor analysis, 
            and industry trends. You provide detailed, data-driven insights. Use web search 
            to find current market information and support your analysis with specific data points.""",
            tools=search_tools + [swarm]
        )
        
        # Strategic Advisor - focuses on actionable recommendations  
        strategic_advisor = Agent(
            system_prompt="""You are a Strategic Business Advisor with 15 years of experience. 
            You take market research and translate it into clear, actionable business strategies. 
            You focus on practical recommendations for positioning, pricing, marketing, and growth. 
            You can also do additional research if needed.""",
            tools=search_tools
        )
        
        return market_analyst, strategic_advisor

# Create our team
analyst, strategist = create_strategy_team()

if analyst and strategist:
    print("✅ Strategy team assembled!")
    print("👥 Team members:")
    print("   🔍 Market Analyst - Data and research specialist")
    print("   💡 Strategic Advisor - Strategy and recommendations expert")


In [ ]:
# Test our specialized team workflow
def test_strategy_workflow():
    """
    Test how our specialized agents work together in sequence
    """
    if not analyst or not strategist:
        print("⚠️ Strategy team not available")
        return
        
    startup_idea = "A subscription service for personalized meal kits using AI to recommend recipes based on dietary preferences, health goals, and local ingredient availability"
    
    print(f"🚀 Startup Idea: {startup_idea}")
    print("\n" + "="*60)
    print("STEP 1: Market Analysis")
    print("="*60)
    
    # Step 1: Market Analyst does research
    analyst_task = f"""
    Conduct comprehensive market research for this startup idea: {startup_idea}
    
    Research and analyze:
    1. Market size and growth trends for meal kit services
    2. Main competitors and their positioning
    3. Customer segments and preferences
    4. Technology trends in food personalization
    
    Use a swarm of agents to gather comprehensive data.
    """
    
    market_research = analyst(analyst_task)
    print("🔍 Market Analysis Complete!")
    print(market_research[:500] + "..." if len(market_research) > 500 else market_research)
    
    print("\n" + "="*60)
    print("STEP 2: Strategy Development") 
    print("="*60)
    
    # Step 2: Strategic Advisor creates strategy
    strategy_task = f"""
    Based on this market research, develop a comprehensive business strategy 
    for the startup idea: {startup_idea}
    
    Market Research Findings:
    {market_research}
    
    Provide:
    1. Positioning strategy
    2. Target customer segments  
    3. Competitive differentiation
    4. Go-to-market recommendations
    5. Key success metrics to track
    """
    
    business_strategy = strategist(strategy_task)
    print("💡 Strategy Development Complete!")
    print(business_strategy)
    
    return market_research, business_strategy

# Run the workflow
if analyst and strategist:
    research_results, strategy_results = test_strategy_workflow()


In [ ]:
# Import file operation tools
from strands_tools import file_write, editor

# Create content creation team
content_writer = Agent(
    system_prompt="""You are a Creative Marketing Writer with expertise in startup marketing.
    You excel at creating compelling marketing copy, campaign ideas, and promotional content.
    Transform business strategies into engaging narratives that resonate with target audiences.
    Focus on benefits, emotional appeal, and clear calls to action."""
)

report_writer = Agent(
    system_prompt="""You are a Professional Report Writer who creates executive-level documents.
    Take research and strategy outputs and format them into clean, professional markdown reports.
    Use proper structure, headings, bullet points, and formatting for easy reading.
    Save all reports using the file_write tool.""",
    tools=[file_write, editor]  # This agent can create and edit files!
)

print("✅ Content creation team ready!")
print("👥 New team members:")
print("   ✍️ Content Writer - Marketing copy and campaigns")
print("   📄 Report Writer - Professional documentation")


In [ ]:
# Test content creation and file operations
def test_content_creation():
    """
    Test how we can create marketing content and save reports
    """
    # Use some sample strategy data (in real scenario, this comes from previous steps)
    sample_strategy = """
    Business Strategy for AI-Powered Meal Kit Service:
    
    1. Target Market: Health-conscious millennials and Gen Z (ages 25-40)
    2. Positioning: "Personalized nutrition made simple with AI"
    3. Key Differentiators: AI recipe recommendations, local sourcing, dietary customization
    4. Go-to-Market: Start with urban areas, partner with fitness apps and health influencers
    5. Revenue Model: Subscription tiers from $15-50/week based on meal frequency
    """
    
    print("📝 Creating marketing content...")
    
    # Step 1: Content Writer creates marketing materials
    content_task = f"""
    Based on this business strategy, create compelling marketing content:
    {sample_strategy}
    
    Create:
    1. A catchy tagline and value proposition
    2. Key marketing messages for different channels
    3. A draft for a social media campaign
    4. An elevator pitch for investors
    """
    
    marketing_content = content_writer(content_task)
    print("✍️ Marketing content created!")
    print("\n📋 Marketing Materials:")
    print("-" * 40)
    print(marketing_content[:400] + "..." if len(marketing_content) > 400 else marketing_content)
    
    print("\n📄 Creating professional report...")
    
    # Step 2: Report Writer formats and saves everything
    report_task = f"""
    Create a comprehensive startup advisory report that includes:
    
    Strategy Summary:
    {sample_strategy}
    
    Marketing Materials:
    {marketing_content}
    
    Format this as a professional markdown report with proper headings and structure.
    Save it as 'startup_advisory_report.md' in the current directory.
    """
    
    report_result = report_writer(report_task)
    print("📊 Professional report generated and saved!")
    print(f"💾 Report result: {report_result}")
    
    return marketing_content

# Test the content creation
marketing_materials = test_content_creation()


In [ ]:
# Check what files were created
import glob
import os

print("📁 Checking for generated files...")
markdown_files = glob.glob("*.md")

for file in markdown_files:
    if file != "README.md":  # Skip the existing README
        print(f"✅ Found: {file}")
        
        # Show file preview
        if os.path.exists(file):
            try:
                with open(file, 'r') as f:
                    content = f.read()
                    print(f"\n📄 Preview of {file}:")
                    print("-" * 50)
                    print(content[:300] + "..." if len(content) > 300 else content)
                    print("-" * 50)
            except Exception as e:
                print(f"❌ Error reading {file}: {e}")

if not any(f != "README.md" for f in markdown_files):
    print("ℹ️ No new files generated (this is normal if file operations aren't working)")


In [ ]:
def complete_startup_advisory(startup_description):
    """
    Complete end-to-end startup advisory workflow
    """
    print("🚀 COMPLETE STARTUP ADVISORY SYSTEM")
    print("=" * 60)
    print(f"📋 Startup Idea: {startup_description}")
    
    if not search_client or not analyst or not strategist:
        print("⚠️ Full system not available - missing search capabilities")
        return
    
    # PHASE 1: Market Research
    print("\n🔍 PHASE 1: Market Research")
    print("-" * 40)
    
    research_task = f"""
    Conduct comprehensive market research for: {startup_description}
    
    Use a swarm of agents to research:
    1. Market size and growth potential
    2. Competitive landscape and key players
    3. Target customer segments and needs
    4. Industry trends and opportunities
    5. Potential challenges and barriers
    """
    
    market_research = analyst(research_task)
    print("✅ Market research completed")
    
    # PHASE 2: Strategy Development  
    print("\n💡 PHASE 2: Strategy Development")
    print("-" * 40)
    
    strategy_task = f"""
    Develop a comprehensive business strategy based on this research:
    
    Startup: {startup_description}
    Research: {market_research}
    
    Provide detailed recommendations for:
    1. Market positioning and value proposition
    2. Target customer segments and personas
    3. Competitive differentiation strategy
    4. Go-to-market plan and channels
    5. Revenue model and pricing strategy
    6. Key milestones and success metrics
    """
    
    business_strategy = strategist(strategy_task)
    print("✅ Business strategy developed")
    
    # PHASE 3: Marketing Content
    print("\n✍️ PHASE 3: Marketing Content Creation")
    print("-" * 40)
    
    content_task = f"""
    Create compelling marketing materials for: {startup_description}
    
    Based on strategy: {business_strategy}
    
    Develop:
    1. Brand positioning and tagline
    2. Key value propositions for different audiences
    3. Marketing messages for various channels
    4. Social media campaign concepts
    5. Investor pitch summary
    6. Customer testimonial framework
    """
    
    marketing_content = content_writer(content_task)
    print("✅ Marketing content created")
    
    # PHASE 4: Professional Report
    print("\n📄 PHASE 4: Report Generation")
    print("-" * 40)
    
    report_task = f"""
    Create a comprehensive startup advisory report including all our work:
    
    Startup: {startup_description}
    Market Research: {market_research}
    Business Strategy: {business_strategy}
    Marketing Content: {marketing_content}
    
    Format as a professional executive report with:
    - Executive Summary
    - Market Analysis
    - Strategic Recommendations  
    - Marketing Plan
    - Implementation Roadmap
    
    Save as 'complete_startup_advisory_report.md'
    """
    
    final_report = report_writer(report_task)
    print("✅ Professional report generated and saved")
    
    print("\n🎉 STARTUP ADVISORY COMPLETE!")
    print("📋 Deliverables generated:")
    print("   • Market research and analysis")
    print("   • Business strategy and recommendations") 
    print("   • Marketing content and campaigns")
    print("   • Professional advisory report")
    
    return {
        'research': market_research,
        'strategy': business_strategy, 
        'marketing': marketing_content,
        'report': final_report
    }

# Test the complete system
test_startup = "A mobile app that uses AR to help people learn cooking by providing real-time visual guidance and ingredient recognition"

if search_client:
    results = complete_startup_advisory(test_startup)
else:
    print("⚠️ Complete system demo requires Perplexity API key")
    print("🔧 Set your PERPLEXITY_API_KEY to see the full workflow in action!")


In [ ]:
# 🎯 YOUR STARTUP IDEA HERE
# Modify this with your own startup concept!

your_startup_idea = """
A platform that connects remote workers with local co-working spaces 
and provides AI-powered matching based on work style, amenities needed, 
and social preferences.
"""

print("💡 Your Startup Idea:")
print(your_startup_idea)
print("\n🤔 Want to analyze this idea? Uncomment the line below:")
print("# your_results = complete_startup_advisory(your_startup_idea)")

# Uncomment this line to run analysis on your idea:
# your_results = complete_startup_advisory(your_startup_idea)
